In [ ]:
import os
from retrieve import extract_tar, load_csv, split_dataset_by_id, DOWNLOAD_ROOT
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# extract_tar(HOUSING_URL, HOUSING_PATH, "housing.tgz")
housing = load_csv(HOUSING_PATH, "housing.csv")
housing.head()

In [ ]:
housing.tail()

In [ ]:
housing.describe()

In [ ]:
import matplotlib.pyplot as plt

housing.hist(bins=50, figsize=(25,20))
plt.show()

# Upon evalutation, the income dataset has clearly been processed. It is 

In [ ]:
housing = housing.reset_index()
train_set, test_set = split_dataset_by_id(housing, 0.2)

In [ ]:
                                   # Shifts left by 3
housing["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_dataset_by_id(housing, 0.2, "id")

In [ ]:
                                                            # No significance behind 42 other than it being the answer
                                                            # to the ultimate Question of Life, the Universe, and
                                                            # Everything
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
# Stratified sampling

# Theoretically, median income is a very important data point
# Below is a histogram split into 5 equal sizes
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0,1.5,3.0,4.5,6.0, np.inf],
                               labels=[1, 2, 3, 4, 5])
housing["income_cat"].hist()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt

# Perform stratified sampling
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Ensure split worked properly
verify_test_sample = pd.DataFrame()
verify_test_sample["strat"] = strat_test_set["income_cat"].value_counts()/len(strat_test_set)
verify_test_sample["all"] = housing["income_cat"].value_counts()/len(housing)
verify_test_sample["rand"] = test_set["median_income"].value_counts()/len(test_set)

verify_test_sample

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
    
# End stratified sampling

In [ ]:
housing = strat_train_set.copy()

housing.plot(kind="scatter", x="longitude", y="latitude")

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
            s=housing["population"]/100, label="population", figsize=(10,7),
            c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()

In [ ]:
# (Pearson's r) Correlation matrix
corr_matrix = housing.corr()

corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

# Income appears to be the best attribute to predict median house value

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

# Horizontal lines are showing up around 500k, 450k, 350k, 220k, 180k?, 110k?
# These appear to be data quirks that should be removed

In [ ]:
# Create new (possibly) correlated attributes

housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
attributes = ["median_house_value", "rooms_per_household", "bedrooms_per_room", "population_per_household"]
scatter_matrix(housing[attributes], figsize=(12, 8))

In [ ]:
# Preparing the data for machine learning

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

# Meidan can only be computed on numerical attributes so other data must be filled another way (or dropped in this case)
housing_num = housing.drop("ocean_proximity", axis=1)

housing_num.median().values

In [ ]:
imputer.fit(housing_num)

X = imputer.transform(housing_num)

housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Algorithms prefer working with numbers rather than categorical values. sklearn provides an ordinal encoder to make
# this conversion simple
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing[["ocean_proximity"]])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# One hot encoding prevents algorithm from thinking ordinal encoded 0's or 1's are close to eachother by assigning
# every category a unique attribute
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat_encoded)
housing_cat_1hot.toarray()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(TransformerMixin, BaseEstimator):
    
    # By adding this hyper parameter, we can more easily automatically test for the best possible strategy
    def __init__(self, add_bedrooms_per_room = True): # With BaseEstimator, args* and kwargs** should not be used
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self # Do nothing except exist
    
    def transform(self, X):
        rooms_per_household = X[:, population_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, households_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

atter_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_atrribs = atter_adder.transform(housing.values)
    

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imuter', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scalar', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
from sklearn.compose import ColumnTransformer

# Use ColumnTransformer to handle numerical columns and categorical columns together in the same pipeline
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
from sklearn.linear_model import LinearRegression

# Create and fit a linear regression model
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

# Holy shit... it's alive!!!
print("Predictions: ", lin_reg.predict(some_data_prepared))
print("Labels: ", list(some_labels))

In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate error levels
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)

# Typical prediction error in dollars
print(lin_rmse)

In [ ]:
# We have underfit the training data. Features do not provide enough information to make good predictions or
# model is not powerful enough

from sklearn.tree import DecisionTreeRegressor

# Attempt to fix underfitting by using a more powerful model
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Calculate error levels
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)

# Literally no error... reaks of overfitting
print(tree_rmse)

In [ ]:
def display_scores(scores):
    print("Validation set scores: ", scores)
    print("Validation set mean: ", scores.mean())
    print("Validation set standard deviation: ", scores.std())

In [ ]:
from sklearn.model_selection import cross_val_score

# Here we perform cross validation on the data by creating ten folds and testing with neg mean squared error
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

# The model clearly performs much worse than it had appeared
display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

display_scores(lin_rmse_scores)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)

print("Training set score: ", forest_rmse)

display_scores(forest_rmse_scores)

In [ ]:
import joblib

# Save model to pickle file so it can be retrieved later without refitting and testing and everything
joblib.dump(forest_rmse, "models/02_end-to-end_machine_learning_project_forest_model.pkl")


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomSearchCV


param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error',
                          return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_
# These are the maximum values so the grid search should be performed again on new, higher values
# I dont actually do this because it would be retesting is very time consuming and I want to move on

In [ ]:
# Analysis of how the best models performed
feature_importance = grid_search.best_estimator_.feature_importances_

extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])

attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importance, attributes), reverse=True)
# This indicates INLAND was the only useful ocean proximity indicator and some attributes like total_bedrooms appear
# to be useless as they are right now

In [ ]:
# It is also important to look at errors the model is making. The book does not go into detail on how to do this

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print("Final RMSE: ", final_rmse)

In [ ]:
# The test data does show how the model generalizes, however a confidence interval would be more revealing

from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                        loc=squared_errors.mean(),
                        scale=stats.sem(squared_errors)))

In [ ]:
from sklearn.svm import SVR

## EXERCISE 1

svr_grid_search =  [
    {'kernel': ["linear"], 'C': [0.01, 0.1, 1, 10, 100]},
    {'kernel': ["rbf"], 'C': [0.01, 0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10, 100]}
]

svr = SVR()

grid_search = GridSearchCV(svr, svr_grid_search, cv=5, scoring='neg_mean_squared_error',
                          return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
best_score = grid_search.best_score_

rmse = np.sqrt(best_score)

# Shit results
print("SVR RMSE: ", rmse)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

## EXERCISE 2 (revised after checking book solution)

param_distribs = {
    'kernel': ['linear', 'rbf'],
    'C': reciprocal(20, 200000),
    'gamma': expon(scale=1.0)
}

svm_reg = SVR()

rand_search = RandomizedSearchCV(svm_reg, param_distributions = param_distribs,
                                 n_iter=50, cv=5 scoring='neg_mean_squared_error',
                                 verbose=2, random_state=42)
                                            # of course random state 42

rand_search.fit(housing_prepared, housing_labels)

In [ ]:
negative_mse = rand_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

In [ ]:
rand_search.best_params_

In [ ]:
# Most good values are smaller than 1 so this may need to be refined
expon_distrib = expon(scale=1.)
samples = expon_distrib.rvs(10000, random_state=42)

plt.figure(figsize=(10,4))
plt.subplot(121)
plt.title("Exponential distribution (scale=1.0)")
plt.hist(samples, bins=50)
plt.subplot(122)
plt.title("Log of this distribution")
plt.hist(np.log(samples), bins=50)
plt.show()

In [ ]:


## EXERCISE 3
